In [1]:
import os
import zipfile
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import keras
import cv2
%matplotlib inline
#import K-Means
from sklearn.cluster import KMeans
# important metrics
from sklearn.metrics import accuracy_score

In [2]:
data = []
label = []
path = "./archive/train/train1/"
c = 0
d = 0
IMG_SIZE = 32
for file in os.listdir(path):
    img=cv2.imread(path+file)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img=img.astype('float32')
    if file[:3]=='cat':
        if c==4: continue
        c+=1
        label.append("cat")
    else:
        if d==4: continue
        d+=1
        label.append("dog")
    data.append(img)
data = np.array(data)

In [3]:
data_label = []
for i in label:
    if i=="cat": data_label.append(0)
    else: data_label.append(1)
data_label = np.array(data_label)

In [4]:
data = data/255.0
reshaped_data = data.reshape(len(data),-1)

In [5]:
reshaped_data.shape = (8,3072)

In [6]:
kmeans = KMeans(n_clusters=2, random_state=0)
clusters = kmeans.fit_predict(reshaped_data)
kmeans.cluster_centers_.shape

(2, 3072)

In [7]:
clusters

array([1, 0, 0, 1, 0, 1, 0, 1])

In [8]:
# 因為K-Means不知道我們的標籤，所以必須根據我們的用法更改K-Means的標籤

# mapping labels from cluster to original labels
def get_reference_dict(clusters,data_label):
    reference_label = {}
    # For loop to run through each label of cluster label
    for i in range(len(np.unique(clusters))):
        index = np.where(clusters == i,1,0)
        num = np.bincount(data_label[index==1]).argmax()
        reference_label[i] = num
    return reference_label

# Mapping predictions to original labels
def get_labels(clusters,refernce_labels):
    temp_labels = np.random.rand(len(clusters))
    for i in range(len(clusters)):
        temp_labels[i] = refernce_labels[clusters[i]]
    return temp_labels

In [9]:
reference_labels = get_reference_dict(clusters,data_label)
predicted_labels = get_labels(clusters,reference_labels)

In [10]:
# Accuracy Score for right predictions
print("準確度:",accuracy_score(predicted_labels,data_label))

準確度: 0.5


In [11]:
def purity_score(y_true,y_pred):
    y_voted_labels = np.zeros(y_true.shape)

    labels = np.unique(y_true)
    ordered_labels = np.arange(labels.shape[0])
    for k in range(labels.shape[0]):
        y_true[y_true==labels[k]] = ordered_labels[k]

    labels = np.unique(y_true)

    bins = np.concatenate((labels, [np.max(labels)+1]), axis=0)

    for cluster in np.unique(y_pred):
        hist, _ = np.histogram(y_true[y_pred==cluster], bins=bins)

        winner = np.argmax(hist)
        y_voted_labels[y_pred==cluster] = winner

    return accuracy_score(y_true, y_voted_labels)

In [12]:
print("純度為:",purity_score(data_label,predicted_labels))

純度為: 0.5
